# Querying

This notebook demonstrates Nexus Forge data [querying features](https://nexus-forge.readthedocs.io/en/latest/interaction.html#querying).

In [ ]:
from kgforge.core import KnowledgeGraphForge

A configuration file is needed in order to create a KnowledgeGraphForge session. A configuration can be generated using the notebook [00-Initialization.ipynb](00%20-%20Initialization.ipynb).

In [ ]:
forge = KnowledgeGraphForge("../../configurations/forge.yml")

## Imports

In [ ]:
from kgforge.core import Resource
from kgforge.specializations.resources import Dataset

## Retrieval

### latest version

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
resource = forge.retrieve(jane.id)

In [ ]:
resource == jane

### specific version

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
forge.register(jane)

In [ ]:
forge.tag(jane, "v1")

In [ ]:
jane.email = "jane.doe@epfl.ch"

In [ ]:
forge.update(jane)

In [ ]:
try:
    # DemoStore
    print(jane._store_metadata.version)
except:
    # BlueBrainNexus
    print(jane._store_metadata._rev)

In [ ]:
jane_v1 = forge.retrieve(jane.id, version=1)

In [ ]:
jane_v1_tag = forge.retrieve(jane.id, version="v1")

In [ ]:
jane_v1 == jane_v1_tag

### crossbucket retrieval
It is possible to retrieve resources stored in buckets different then the configured one. The configured store should of course support it.

In [ ]:
resource = forge.retrieve(jane.id, cross_bucket=True) # cross_bucket defaults to False

### error handling

In [ ]:
resource = forge.retrieve("123")

In [ ]:
resource is None

## Searching

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel. Commented lines are for DemoModel.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [ ]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [ ]:
dataset = Dataset(forge, type="Dataset", contribution=[contribution_jane, contribution_john])
dataset.add_distribution("../../data/associations.tsv")

In [ ]:
forge.register(dataset)

In [ ]:
forge.as_json(dataset)

### Paths as filters

The `paths` method load the data structure for the given type.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about modeling and types.

In [ ]:
p = forge.paths("Dataset")

You have autocompletion on `p` and this can be used to create search filters.

Note: There is a known issue for RdfModel which requires using `p.type.id` instead of `p.type`.

In [ ]:
resources = forge.search(p.type.id=="Dataset", limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

In [ ]:
forge.as_json(resources[2])

In [ ]:
forge.as_dataframe(resources, store_metadata=True)

### Nested property querying

Property autocompletion is available on a path `p` even for nested properties like `p.contribution`.

In [ ]:
# Search for resources of type Dataset and with a attached files of content type text/tab-separated-values
resources = forge.search(p.type.id == "Dataset", p.distribution.encodingFormat == "text/tab-separated-values", limit=3)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

### Dict as filters
A dictionary can be provided for filters:
* {'type': {'id':'Dataset'}} is equivalent to p.type.id=="Dataset"
* only the '==' operator is supported
* nested dict are supported
* it is not mandatory for the provided properties and values to be defined in the forge model. Results will be retrieved if there are corresponding data in the store.

This feature is not supported when using the DemoStore


In [ ]:
# Search for resources of type Dataset and with conribution from agent named "Jane Doe"
filters = {"type": "Dataset", "contribution":{"type":"Contribution", "agent":{"name":"Jane Doe"}}}
resources = forge.search(filters, limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources, store_metadata=True)

### Crossbucket search
It is possible to search for resources stored in buckets different then the configured one. The configured store should of course support it.

In [ ]:
resources = forge.search(p.type.id == "Dataset", limit=3, cross_bucket=True)  # cross_bucket defaults to False

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

In [ ]:
#Furthermore it is possible to filter by bucket when cross_bucket is set to True. Setting a bucket value when cross_bucket is False will trigger a not_supported exception.
resources = forge.search(p.type.id == "Dataset", limit=3, cross_bucket=True, bucket=<str>)  # add a bucket

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
forge.as_dataframe(resources)

## Graph traversing

SPARQL is used as a query language.

A SPARQL query rewriting strategy lets users write simplified queries without using prefix declarations, prefix names or long IRIs. With this strategy, the user could only provides type and property names. For a given entity type, these names could be seen in its template.

Please refer to the [Modeling.ipynb](11%20-%20Modeling.ipynb) notebook to learn about templates.

Note: DemoStore doesn't implement SPARQL operations yet. Please use another store for this section.

Note: DemoModel and RdfModel schemas have not been synchronized yet. This section is to be run with RdfModel.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")
contribution_jane = Resource(type="Contribution", agent=jane)

In [ ]:
john = Resource(type="Person", name="John Smith")
contribution_john = Resource(type="Contribution", agent=john)

In [ ]:
association = Resource(type="Dataset", contribution=[contribution_jane, contribution_john])

In [ ]:
forge.register(association)

In [ ]:
forge.template("Dataset")

### Prefix and namespace free SPARQL query

When a forge RDFModel is configured, then there is no need to provide prefixes and namespaces when writing a SPARQL query. Prefixes and namespaces will be automatically inferred from the provided schemas and/or JSON-LD context and the query rewritten accordingly.

In [ ]:
query = """
    SELECT ?id ?name
    WHERE {
        ?id a Dataset ;
        contribution/agent ?contributor.
        ?contributor name ?name.
    }
"""

In [ ]:
resources = forge.sparql(query, limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

In [ ]:
forge.as_dataframe(resources)

### rewritten query display
The prefix free SPARQL query provided above is rewritten as the ouput of cell when a forge Model is configured. 

In [ ]:
resources = forge.sparql(query, limit=3, debug=True)

### SPARQL query

regular SPARQL query can also be provided.

In [ ]:
query = """
PREFIX dc: <http://purl.org/dc/elements/1.1/>
   PREFIX dcat: <http://www.w3.org/ns/dcat#>
   PREFIX dcterms: <http://purl.org/dc/terms/>
   PREFIX mba: <http://api.brain-map.org/api/v2/data/Structure/>
   PREFIX nsg: <https://neuroshapes.org/>
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX prov: <http://www.w3.org/ns/prov#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX schema: <http://schema.org/>
   PREFIX sh: <http://www.w3.org/ns/shacl#>
   PREFIX shsh: <http://www.w3.org/ns/shacl-shacl#>
   PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
   PREFIX vann: <http://purl.org/vocab/vann/>
   PREFIX void: <http://rdfs.org/ns/void#>
   PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
   PREFIX : <https://neuroshapes.org/>
   SELECT ?id ?name
   WHERE {
       ?id a schema:Dataset ;
       nsg:contribution/prov:agent ?contributor.
       ?contributor schema:name ?name.
   }
"""

In [ ]:
resources = forge.sparql(query, limit=3)

In [ ]:
type(resources)

In [ ]:
len(resources)

In [ ]:
type(resources[0])

In [ ]:
forge.as_dataframe(resources)

## Downloading

Note: DemoStore doesn't implement file operations yet. Please use another store for this section.

In [ ]:
jane = Resource(type="Person", name="Jane Doe")

In [ ]:
! ls -p ../../data | egrep -v /$

In [ ]:
distribution = forge.attach("../../data")

In [ ]:
association = Resource(type="Association", agent=jane, distribution=distribution)

In [ ]:
forge.register(association)

In [ ]:
forge.download(association, "distribution.contentUrl", "./downloaded/")

In [ ]:
! ls -l ./downloaded/

In [ ]:
# ! rm -R ./downloaded/